<a href="https://colab.research.google.com/github/Syilun/face_people_recognization/blob/master/Train_YOLOv4_softlink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 記得先打開GPU
* 編輯 >> 筆記本設定 >> 硬體加速器 --> GPU

## 檢查GPU型號

In [ ]:
# check NVIDIA GPU
! nvidia-smi

Sun Aug 16 16:36:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 下載 Darknet (AlexeyAB 版本)

In [ ]:
! git clone https://github.com/AlexeyAB/darknet.git

fatal: destination path 'darknet' already exists and is not an empty directory.


In [ ]:
!head darknet/Makefile

GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0



# 指定訓練/測試資料
### 在 /cfg 建立 `train.txt` , `test.txt`

In [ ]:
# 若檔案存在則先刪除
! rm {GDRIVE_YOLO_CFG_PATH}/train.txt
! rm {GDRIVE_YOLO_CFG_PATH}/test.txt
! ls -al {GDRIVE_YOLO_CFG_PATH}

rm: cannot remove '{GDRIVE_YOLO_CFG_PATH}/train.txt': No such file or directory
rm: cannot remove '{GDRIVE_YOLO_CFG_PATH}/test.txt': No such file or directory
ls: cannot access '{GDRIVE_YOLO_CFG_PATH}': No such file or directory


In [ ]:
import os

datasets = [GDRIVE_YOLO_DATA_PATH_py + '/' + f for f in os.listdir(GDRIVE_YOLO_DATA_PATH_py) if f.endswith('.jpg')]
split_boundary = int(len(datasets) * 0.8)

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print("訓練資料數：", split_boundary)

In [ ]:
with open(GDRIVE_YOLO_CFG_PATH_py + '/' + 'train.txt', 'w') as f:
    f.write('\n'.join(datasets[0:split_boundary]))

In [ ]:
# 查看 train.txt
! head /content/drive/My\ Drive/people/cfg/train.txt

In [ ]:
! cat /content/drive/My\ Drive/people/yolos/img_6411.txt

In [ ]:
with open(GDRIVE_YOLO_CFG_PATH_py + '/' + 'test.txt', 'w') as f:
    f.write('\n'.join(datasets[split_boundary:]))

In [ ]:
# 查看 test.txt
# ! head /content/drive/My\ Drive/people/cfg/test.txt
! head /people/cfg/test.txt

# 開始訓練

#### 編譯 YOLOv4

In [ ]:
from time import time

start = time()
! cd darknet; make
print("經過時間 = {:4.2f} 秒" .format(time()-start))

#### (Option) 下載 pre-trained weithts

In [ ]:
# 如果是第一次Train才下載此pre-trained weight !!!!!!!!
# 如果要接續之前訓練產生的結果則不必執行
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
! cat /content/drive/My\ Drive/people/cfg/obj-person.data

In [ ]:
! cat /content/cfg/obj-person.data

### 以下擇一執行訓練

In [ ]:
# (1) 使用pre-trained weights - 訓練單一類別 person
! darknet/darknet detector train {GDRIVE_YOLO_CFG_PATH}/obj-person.data {GDRIVE_YOLO_CFG_PATH}/yolov4-person.cfg yolov4.conv.137 -dont_show | grep "avg loss"

# (2) 使用pre-trained weights - 訓練17種類別
#! darknet/darknet detector train {GDRIVE_YOLO_CFG_PATH}/obj-cat17.data {GDRIVE_YOLO_CFG_PATH}/yolov4-cat17.cfg yolov4.conv.137 -dont_show | grep "avg loss"

# (3) 使用最後一次訓練產生的weights - 訓練單一類別 person
#! darknet/darknet detector train {GDRIVE_YOLO_CFG_PATH}/obj-person.data {GDRIVE_YOLO_CFG_PATH}/yolov4-person.cfg {GDRIVE_WEIGHTS_PATH}/yolov4-person_last.weights -dont_show | grep "avg loss"

# (4) 使用最後一次訓練產生的weights - 訓練17種類別
#! darknet/darknet detector train {GDRIVE_YOLO_CFG_PATH}/obj-cat17.data {GDRIVE_YOLO_CFG_PATH}/yolov4-cat17.cfg {GDRIVE_WEIGHTS_PATH}/yolov4-cat17_last.weights -dont_show | grep "avg loss"